# Prompt Template으로 배포한 AI Service 사용하기
Prompt Template을 사용하여 AI Service를 배포한 후, 해당 서비스를 사용하는 방법에 대해 설명합니다.


## 라이브러리 설치


In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

!pip install ibm-watsonx-ai==1.3.6

## 환경정보 설정

In [ ]:
# IBM Cloud API Key와 URL 설정
api_key = "<CLOUD_API_KEY>"
# region에 따라 주소가 다를 수 있습니다. 주소를 확인해 주세요.
ibm_cloud_url = "https://us-south.ml.cloud.ibm.com" 


# 배포한 프롬프트 템플릿 모델의 AI Service URL
ibm_ai_service_url = "<IBM_AI_SERVICE_URL>"


번역할 원문을 입력합니다.

In [ ]:
user_input = "진짜 강한 힘은 나를 넘어 ‘우리’를 생각할 때 나온다. 더 큰 가치를 위해 싸운 사람만이 진짜 세상을 바꿀 수 있다"
# user_input = "성공은 열정을 잃지 않고 실패를 거듭할 수 있는 능력이다."

배포한 AI Service를 호출하여 처리 결과를 확인합니다.

In [ ]:
import requests
import json

# IBM Cloud API Key를 사용하여 토큰 발급
token_response = requests.post(
    'https://iam.cloud.ibm.com/identity/token',
    data={
        "apikey": api_key,
        "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'
    }
)
mltoken = token_response.json().get("access_token")
if not mltoken:
    raise Exception("토큰 발급 실패!")

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {mltoken}',
    'Accept': 'text/event-stream'  # 스트리밍 응답 처리용 헤더
}

payload_scoring = {
    "parameters": 
        {
            "prompt_variables": 
            {
                "context" : user_input
            }
        }
    
}

response = requests.post(
    ibm_ai_service_url,
    headers=headers,
    json=payload_scoring,
    stream=True
)


print("=== 응답 결과 ===")
# print(response.json())
# generated_text 추출
generated_text = response.json()['results'][0]['generated_text']

# 첫 줄만 추출
first_line = generated_text.split('\n\nInput')[0]

# 결과 출력
print("원문 : " + user_input)
print("번역 :" + first_line)
